***Import Libraries***

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import datetime as dt
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



***Data Cleaning***


> In this step, I downloaded the dataset from my original GitHub repository, extracted the columns that I wanted to analyze.



In [ ]:
#import datasets from GitHub
url_med = 'https://raw.githubusercontent.com/vchen19/baltimore_walk_score_and_income/main/Median_Household_Income.csv'
url_walk = 'https://raw.githubusercontent.com/vchen19/baltimore_walk_score_and_income/main/Walk_Score.csv'
url_food = 'https://raw.githubusercontent.com/vchen19/baltimore_food_deserts/main/Average_Healthy_Food_Availability_Index.csv'
url_life = 'https://raw.githubusercontent.com/vchen19/baltimore_food_deserts/main/Life_Expectancy.csv'
#read datasets into pandas dataframes
df_med = pd.read_csv(url_med)
df_walk = pd.read_csv(url_walk)
df_food = pd.read_csv(url_food)
df_life = pd.read_csv(url_life)
#extract relevant columns- household income
df_med = pd.DataFrame(df_med, columns = ['mhhi11'])
#make column take on the index in the new dataset, not the old
df_med = df_med.reset_index(drop=True) 
#extract relevant columns- walk score
df_walk = pd.DataFrame(df_walk, columns = ['OBJECTID', 'CSA2010',	'wlksc11'])
df_walk = df_walk.reset_index(drop=True)
#extract relevant columns- healthy food availability index
df_food = pd.DataFrame(df_food, columns = ['hfai12'])
df_food = df_food.reset_index(drop=True)
#extract relevant columns- life expectancy
df_life = pd.DataFrame(df_life, columns = ['lifexp11'])
#combine datasets into one big dataset
df = pd.concat([df_walk, df_med], axis=1)
df = pd.concat([df, df_food], axis = 1)
df = pd.concat([df,df_life], axis = 1)
df = df.reset_index(drop=True) 



**Plotting using Plotly Express**


> In this step, I plotted density of fast food restaurants against walk score with the size of the point corresponding to median household income. This allowed me to visually analyze whether there were any interesting trends between all of these variables.



In [ ]:
fig = px.scatter(df, x="wlksc11", y="lifexp11",size="mhhi11", title = "2011 Walk Score, Median Household Income, and Life Expectancy")
fig.show()

fig = px.scatter(df, x="wlksc11", y="lifexp11",size="hfai12", title = "2011/2012 Walk Score, Health Food Availability Index, and Life Expectancy")
#fig.update_xaxes(range=[0, 5])
fig.show()

fig = px.scatter(df, x="mhhi11", y="lifexp11",size="hfai12", title = "2011/2012 Median Household Income, Health Food Availability Index, and Life Expectancy")
fig.show()



**Multiple Regression**


> In this step, I ran multiple regression to predict a neighborhood's healthy food availability index with median household income and walk score.



In [ ]:
Y = df['lifexp11']
X = df[['wlksc11', 'mhhi11', 'hfai12']]
X = sm.add_constant(X) # adding a constant
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:               lifexp11   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.322
Method:                 Least Squares   F-statistic:                     9.555
Date:                Fri, 30 Apr 2021   Prob (F-statistic):           4.12e-05
Time:                        03:36:26   Log-Likelihood:                -160.05
No. Observations:                  55   AIC:                             328.1
Df Residuals:                      51   BIC:                             336.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         68.6400      2.627     26.128      0.0